In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
from sklearn import linear_model
# from __future__ import division

%matplotlib inline
seaborn.set()
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False

G:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps,
G:\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:597: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  eps=np.finfo(np.float).eps, copy_X=True, fit_path=True,
G:\Anaconda3\lib\site

In [2]:
# 确诊数据导入和初步处理
f_confirm = open(r'各国R0.csv',encoding = "utf-8")
data_confirm = pd.read_csv(f_confirm)
# 处理时间格式
data_confirm['时间'] = pd.to_datetime(data_confirm['时间'])

In [3]:
# 政策数据导入和初步处理
f_policy = open(r'OxCGRT_latest.csv',encoding='utf-8')
data_policy = pd.read_csv(f_policy)
# 处理时间格式
data_policy['Date'] = data_policy['Date'].astype(str)
data_policy['Date'] = pd.to_datetime(data_policy['Date'])
# 只取国家级别的政策信息
data_policy = data_policy[data_policy['Jurisdiction'] == 'NAT_TOTAL']

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py:3156: DtypeWarning: Columns (2,3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# 对政策指标的二值化更改
# 封闭措施
# C1
data_policy['C1'] = 0
data_policy['C1'][data_policy['C1_School closing']<=1.0] = 0
data_policy['C1'][data_policy['C1_School closing']>=2.0] = 1
# C2
data_policy['C2'] = 0
data_policy['C2'][data_policy['C2_Workplace closing']<=1.0] = 0
data_policy['C2'][data_policy['C2_Workplace closing']>=2.0] = 1
# C3
data_policy['C3'] = 0
data_policy['C3'][data_policy['C3_Cancel public events']>=1.0] = 1
# C4
data_policy['C4'] = 0
data_policy['C4'][data_policy['C4_Restrictions on gatherings']<=3.0] = 0
data_policy['C4'][data_policy['C4_Restrictions on gatherings']>=4.0] = 1
# C5
data_policy['C5'] = 0
data_policy['C5'][data_policy['C5_Close public transport']<=1.0] = 0
data_policy['C5'][data_policy['C5_Close public transport']>=2.0] = 1
# C6
data_policy['C6'] = 0
data_policy['C6'][data_policy['C6_Stay at home requirements']<=1.0] = 0
data_policy['C6'][data_policy['C6_Stay at home requirements']>=2.0] = 1
# C7
data_policy['C7'] = 0
data_policy['C7'][data_policy['C7_Restrictions on internal movement']<=1.0] = 0
data_policy['C7'][data_policy['C7_Restrictions on internal movement']>=2.0] = 1
# C8
data_policy['C8'] = 0
data_policy['C8'][data_policy['C8_International travel controls']<=1.0] = 0
data_policy['C8'][data_policy['C8_International travel controls']>=2.0] = 1

# 经济政策
# E1
data_policy['E1'] = 0
data_policy['E1'][data_policy['E1_Income support']>=1.0] = 1
# E2
data_policy['E2'] = 0
data_policy['E2'][data_policy['E2_Debt/contract relief']<=1.0] = 0
data_policy['E2'][data_policy['E2_Debt/contract relief']>=2.0] = 1
# E3
data_policy['E3'] = 0
data_policy['E3'][data_policy['E3_Fiscal measures']<1e+07] = 0
data_policy['E3'][data_policy['E3_Fiscal measures']>=1e+07] = 1
# E4
data_policy['E4'] = 0
data_policy['E4'][data_policy['E4_International support']<1e+05] = 0
data_policy['E4'][data_policy['E4_International support']>=1e+05] = 1

# 卫生系统政策
# H1
data_policy['H1'] = 0
data_policy['H1'][data_policy['H1_Public information campaigns']<=1.0] = 0
data_policy['H1'][data_policy['H1_Public information campaigns']>=2.0] = 1
# H2
data_policy['H2'] = 0
data_policy['H2'][data_policy['H2_Testing policy']<=1.0] = 0
data_policy['H2'][data_policy['H2_Testing policy']>=2.0] = 1
# H3
data_policy['H3'] = 0
data_policy['H3'][data_policy['H3_Contact tracing']<=1.0] = 0
data_policy['H3'][data_policy['H3_Contact tracing']>=2.0] = 1
# H4
data_policy['H4'] = 0
data_policy['H4'][data_policy['H4_Emergency investment in healthcare']<1e+06] = 0
data_policy['H4'][data_policy['H4_Emergency investment in healthcare']>=1e+06] = 1
# H5
data_policy['H5'] = 0
data_policy['H5'][data_policy['H5_Investment in vaccines']<1e+06] = 0
data_policy['H5'][data_policy['H5_Investment in vaccines']>=1e+06] = 1
# H6
data_policy['H6'] = 0
data_policy['H6'][data_policy['H6_Facial Coverings']<=2.0] = 0
data_policy['H6'][data_policy['H6_Facial Coverings']>=3.0] = 1
# H7
data_policy['H7'] = 0
data_policy['H7'][data_policy['H7_Vaccination policy']<=3.0] = 0
data_policy['H7'][data_policy['H7_Vaccination policy']>=4.0] = 1

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\zay\AppD

In [5]:
# 建立字典，对应政策代号及其中英文名
dic_policyEng = dict()
dic_policyChi = dict()

# 封闭措施
dic_policyEng['C1'] = 'School closing'
dic_policyChi['C1'] = '关闭学校'
dic_policyEng['C2'] = 'Workplace closing'
dic_policyChi['C2'] = '关闭工作场所'
dic_policyEng['C3'] = 'Cancel public events'
dic_policyChi['C3'] = '取消公共活动'
dic_policyEng['C4'] = 'Restrictions on gatherings'
dic_policyChi['C4'] = '限制集会'
dic_policyEng['C5'] = 'Close public transport'
dic_policyChi['C5'] = '关闭公共交通'
dic_policyEng['C6'] = 'Stay at home requirements'
dic_policyChi['C6'] = '居家要求'
dic_policyEng['C7'] = 'Restrictions on internal movement'
dic_policyChi['C7'] = '国内移动限制'
dic_policyEng['C8'] = 'International travel controls'
dic_policyChi['C8'] = '国际旅行控制'

# 经济政策
dic_policyEng['E1'] = 'Income support'
dic_policyChi['E1'] = '收入支持'
dic_policyEng['E2'] = 'Debt/contract relief'
dic_policyChi['E2'] = '债务/合同放宽'
dic_policyEng['E3'] = 'Fiscal measures'
dic_policyChi['E3'] = '财政措施'
dic_policyEng['E4'] = 'International support'
dic_policyChi['E4'] = '给予国际支持'

# 卫生系统政策
dic_policyEng['H1'] = 'Public information campaigns'
dic_policyChi['H1'] = '公共信息宣传'
dic_policyEng['H2'] = 'Testing policy'
dic_policyChi['H2'] = '检测政策'
dic_policyEng['H3'] = 'Contact tracing'
dic_policyChi['H3'] = '接触者追踪'
dic_policyEng['H4'] = 'Emergency investment in healthcare'
dic_policyChi['H4'] = '医疗保健紧急投资'
dic_policyEng['H5'] = 'Investment in vaccines'
dic_policyChi['H5'] = '疫苗投资'
dic_policyEng['H6'] = 'Facial Coverings'
dic_policyChi['H6'] = '口罩政策'
dic_policyEng['H7'] = 'Vaccination Policy'
dic_policyChi['H7'] = '疫苗接种政策'

ls_code = list(dic_policyEng.keys())
for code in ['E3','E4','H4','H5']:
    ls_code.remove(code)
ls_code

['C1',
 'C2',
 'C3',
 'C4',
 'C5',
 'C6',
 'C7',
 'C8',
 'E1',
 'E2',
 'H1',
 'H2',
 'H3',
 'H6',
 'H7']

In [6]:
# 此函数用于给出政策是否引入和取消的指标
def genFlags(policySeries):
    Length = policySeries.shape[0]
    introduceSeries = pd.Series(np.zeros(Length))
    liftSeries = pd.Series(np.zeros(Length))
    if policySeries.iloc[0] == 1:
        introduceSeries.iloc[0] = 1
    else:
        introduceSeries.iloc[0] = 0
    liftSeries.iloc[0] = 0
    for i in range(1,Length):
        if policySeries.iloc[i-1] == 0 and policySeries.iloc[i] == 1:   # 政策从0变为1，则introduce指标变为1
            introduceSeries.iloc[i] = 1
        elif policySeries.iloc[i-1] ==1 and policySeries.iloc[i] == 0: # 政策从1变为0，则lift指标变为1
            liftSeries.iloc[i] = 1
        else:
            introduceSeries.iloc[i] = 0
            liftSeries.iloc[i] = 0
    return introduceSeries,liftSeries

In [7]:
# 提取主要信息
data_brief = data_policy[['CountryName','Date']]
for code in ls_code:
    data_brief[code] = data_policy[code]
data_brief

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,CountryName,Date,C1,C2,C3,C4,C5,C6,C7,C8,E1,E2,H1,H2,H3,H6,H7
0,Aruba,2020-01-01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Aruba,2020-01-02,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Aruba,2020-01-03,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Aruba,2020-01-04,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Aruba,2020-01-05,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110428,Zimbabwe,2021-01-23,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
110429,Zimbabwe,2021-01-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
110430,Zimbabwe,2021-01-25,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
110431,Zimbabwe,2021-01-26,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 当天有无政策
data_brief['policyExists'] = 0
for code in ls_code:
    data_brief['policyExists'] += data_brief[code]
data_brief['policyExists'][data_brief['policyExists']>1] = 1

# 对每个政策判断其是否引入和取消，28天内有无该政策
data_prepare = pd.DataFrame()
ls_country = list(data_brief['CountryName'].drop_duplicates())
for country in ls_country:
    subdata = data_brief[data_brief['CountryName']==country]
    subdata = subdata.reset_index(drop=True)
    Length = subdata.shape[0]

    for code in ls_code:
        # 政策是否引入或放宽
        introduceSeries,liftSeries = genFlags(subdata[code])
        introduceSeries = introduceSeries.reset_index(drop=True)
        liftSeries = liftSeries .reset_index(drop=True)
        subdata[code+'Introduce'] = introduceSeries
        subdata[code+'Lift'] = liftSeries
        
        # 28天内有无该政策
        subdata[code+'28'] = 0
        for i in range(Length):
            end = min(i+28,Length)
            subdata[code+'28'].iloc[i] = subdata[code].iloc[i:end].sum()
        
    # 28天内都没有任何政策
    subdata['Policies28'] = 0
    for i in range(Length):
        end = min(i+28,Length)
        subdata['Policies28'].iloc[i] = subdata['policyExists'].iloc[i:end].sum()

    startDate = pd.to_datetime('2020-01-22')
    endDate = pd.to_datetime('2020-12-31')
    subdata = subdata[(subdata['Date']>=startDate)&(subdata['Date']<=endDate)]
    data_prepare = data_prepare.append(subdata)
data_brief = data_prepare.copy()

In [10]:
# 国家名统一
namesOrigin = ['Czechia','Korea, South','US']
namesNew = ['Czech Republic','South Korea','United States']
for i in range(len(namesOrigin)):
    nameOrigin = namesOrigin[i]
    nameNew = namesNew[i]
    data_confirm['国家英文名'][data_confirm['国家英文名']==nameOrigin] = nameNew

data_brief['CountryName'][data_brief['CountryName']=='Democratic Republic of Congo'] = 'Congo (Kinshasa)'
data_brief['CountryName'][data_brief['CountryName']=='Congo'] = 'Congo (Brazzaville)'
data_brief['CountryName'][data_brief['CountryName']=='Slovak Republic'] = 'Slovakia'


C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove t

In [14]:
ls_countryPolicy = list(data_brief['CountryName'].drop_duplicates())
ls_countryConfirm = list(data_confirm['国家英文名'].drop_duplicates())
ls_dropCountry = []
for country in ls_countryPolicy:
    if country not in ls_countryConfirm:
        print(country)
        ls_dropCountry.append(country)

for country in ls_dropCountry:
    ls_countryPolicy.remove(country)

Aruba
Bermuda
Cape Verde
Faeroe Islands
Greenland
Guam
Hong Kong
Kyrgyz Republic
Kiribati
Macao
Myanmar
Puerto Rico
Palestine
Turkmenistan
Tonga
Taiwan
United States Virgin Islands


In [15]:
# 加入R0信息
data_prepare = pd.DataFrame()
for country in ls_countryPolicy:
    subdata_policy = data_brief[data_brief['CountryName']==country]
    subdata_policy['R0'] = 0
    subdata_confirm = data_confirm[data_confirm['国家英文名']==country]
    SeriesR0 = subdata_confirm['R0'][(subdata_confirm['时间']>=startDate)&(subdata_confirm['时间']<=endDate)]
    SeriesR0 = SeriesR0.reset_index(drop=True)
    subdata_policy = subdata_policy.reset_index(drop=True)
    subdata_policy['R0'] = SeriesR0
    data_prepare = data_prepare.append(subdata_policy)

data_brief = data_prepare.copy()

C:\Users\zay\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [16]:
data_brief.to_csv('政策详细信息_new.csv',index=False,sep=',',encoding="utf_8_sig")

In [17]:
data_confirm.to_csv('各国R0_国家名统一.csv',index=False,sep=',',encoding="utf_8_sig")

In [18]:
data_brief['CountryName'].drop_duplicates().shape[0]

168

# 时间切片 每个政策引入为第一天


# 准备时间切片
# 新的data_prepare包含：国家名称、指标值、是否引入或取消政策、R0值
ls_policy = ['C1_School closing','C2_Workplace closing','C3_Cancel public events','C4_Restrictions on gatherings','C5_Close public transport','C6_Stay at home requirements','C7_Restrictions on internal movement','C8_International travel controls']
data_prepare = pd.DataFrame()
for name in countryNamesChi:
    nameEng = countryNamesDict[name]
    data_country = data_policy[data_policy['CountryName']==nameEng]
    data_country = data_country[data_country['Date']<='2020-09-27']
    data_country = data_country[data_country['Date']>='2020-01-22']
    data_country = data_country.iloc[:250,:]
    
    # 把所需的信息放入data_countryBrief
    data_countryBrief = pd.DataFrame()
    # 国家名
    data_countryBrief['CountryName'] = data_country['CountryName'].reset_index(drop=True)
    data_countryBrief['CountryName'] = name
    # 时间
    data_countryBrief['Date'] = data_country['Date'].reset_index(drop=True)
    Length = data_countryBrief.shape[0]
    # 单个政策信息
    for i in range(8):
        policySeries = data_country[ls_policy[i]].reset_index(drop=True)
        data_countryBrief[ls_policy[i]] = policySeries
        IntroduceSeries,liftSeries = genFlags(policySeries)
        head1 = 'C{}_Introduce'.format(i+1)
        head2 = 'C{}_Lift'.format(i+1)
        data_countryBrief[head1] = IntroduceSeries
        data_countryBrief[head2] = liftSeries
    # 复合政策信息
    data_countryBrief['Multiple_Introduce'] = 0 # 是否有多个政策引入
    data_countryBrief['Multiple_Lift'] = 0      # 是否有多个政策取消
    data_countryBrief['Multiple_Policy'] = 0    # 是否有多个政策同时引入或取消
    data_countryBrief['Introduces'] = 0         # 引入指标之和
    data_countryBrief['Lifts'] = 0              # 取消指标之和
    data_countryBrief['Policies'] = 0           # 政策指标之和
    data_countryBrief['Policies28'] = 0         # 从这一天开始后28天政策指标之和
    for i in range(8):
        data_countryBrief['Introduces'] += data_countryBrief['C{}_Introduce'.format(i+1)]
        data_countryBrief['Lifts'] += data_countryBrief['C{}_Lift'.format(i+1)]
    data_countryBrief['Policies'] = data_countryBrief['Introduces'] + data_countryBrief['Lifts']
    data_countryBrief['Multiple_Policy'][data_countryBrief['Policies']>1] = 1
    data_countryBrief['Multiple_Introduce'][data_countryBrief['Introduces']<=1] = 0
    data_countryBrief['Multiple_Introduce'][data_countryBrief['Introduces']>1] = 1
    data_countryBrief['Multiple_Lift'][data_countryBrief['Lifts']<=1] = 0
    data_countryBrief['Multiple_Lift'][data_countryBrief['Lifts']>1] = 1
    for i in range(Length):
        end = min(i+27,Length-1)
        data_countryBrief['Policies28'].iloc[i] = data_countryBrief['Policies'].iloc[i:end].sum()
    
    # 政策是否发生改变
    data_countryBrief['PolicyChanges'] = 0
    for i in range(1,Length):
        s_bool = 0 
        for string in ls_policy:
            s_bool += abs(data_countryBrief[string].iloc[i] - data_countryBrief[string].iloc[i-1])
        if s_bool != 0:
            data_countryBrief['PolicyChanges'].iloc[i] = 1
    # R0
    data_countryBrief['R0'] = data_confirm[data_confirm['国家']==name]['疫情状态参数'].reset_index(drop=True)
    subdata = data_countryBrief[data_countryBrief['R0'] != '#REF!']
    subdata['R0'] = subdata['R0'].astype('float')
    ls_index = list(subdata[subdata['R0'] < 0].index)
    for index in ls_index:
        if index + 1 not in ls_index and index - 1 not in ls_index:
            data_countryBrief['R0'].loc[index] = 0.5 * (subdata['R0'].loc[index-1]+subdata['R0'].loc[index+1])
        else:
            data_countryBrief['R0'].loc[index] = 0.5 * (subdata['R0'].loc[index-2]+subdata['R0'].loc[index+2])
    # 把各国信息整合
    data_prepare = data_prepare.append(data_countryBrief,ignore_index=True)
data_prepare.to_csv('data_prepare.csv',index=False,sep=',',encoding="utf_8_sig")

# 把政策改变前一天的数据和改变后28天数据分别放进29个DataFrame表格中
data_day0 = pd.DataFrame()
data_day1 = pd.DataFrame()
data_day2 = pd.DataFrame()
data_day3 = pd.DataFrame()
data_day4 = pd.DataFrame()
data_day5 = pd.DataFrame()
data_day6 = pd.DataFrame()
data_day7 = pd.DataFrame()
data_day8 = pd.DataFrame()
data_day9 = pd.DataFrame()
data_day10 = pd.DataFrame()
data_day11 = pd.DataFrame()
data_day12 = pd.DataFrame()
data_day13 = pd.DataFrame()
data_day14 = pd.DataFrame()
data_day15 = pd.DataFrame()
data_day16 = pd.DataFrame()
data_day17 = pd.DataFrame()
data_day18 = pd.DataFrame()
data_day19 = pd.DataFrame()
data_day20 = pd.DataFrame()
data_day21 = pd.DataFrame()
data_day22 = pd.DataFrame()
data_day23 = pd.DataFrame()
data_day24 = pd.DataFrame()
data_day25 = pd.DataFrame()
data_day26 = pd.DataFrame()
data_day27 = pd.DataFrame()
data_day28 = pd.DataFrame()
ls_data = [data_day0,data_day1,data_day2,data_day3,data_day4,data_day5,data_day6,data_day7,data_day8,data_day9,data_day10,data_day11,data_day12,data_day13,data_day14,data_day15,data_day16,data_day17,data_day18,data_day19,data_day20,data_day21,data_day22,data_day23,data_day24,data_day25,data_day26,data_day27,data_day28]

In [ ]:
# 把每个阶段的第i天分配到data_dayi中
timedelta1 = pd.to_timedelta('7D')
timedelta2 = pd.to_timedelta('27D')
startDate = pd.to_datetime('2020-01-22')
endDate = pd.to_datetime('2020-09-27')
data_day1 = data_prepare[data_prepare['PolicyChanges']==1]      # 政策发生改变时为第一天
data_day1_1 = data_prepare[data_prepare['Policies28']==0]       # 28天内都无政策
data_day1 = data_day1.append(data_day1_1)

# data_day1 = data_day1.iloc[1:,:]                                # 敏感性分析中，各国第一次引入政策不纳入讨论
# data_day1 = data_day1[data_day1['Multiple_Policy']==0]    # 敏感性分析中，只讨论仅改变一个政策的阶段
data_day1 = data_day1[data_day1['Date'] - startDate >= timedelta1]
data_day1 = data_day1[endDate - data_day1['Date'] >= timedelta2] # 保证所选取的一个阶段里的28天都在数据中
# data_day1.to_csv('test1.csv',index=False,sep=',',encoding="utf_8_sig")
index = data_day1.index
index = index-1

for i in range(29):
    newindex = index + i
    newindex = newindex.to_list()
    Length = len(newindex)
    ls_data[i] = data_prepare.loc[newindex,:]
    ls_data[i]['R0'][ls_data[i]['R0']=='#REF!'] = 1
    ls_data[i] = ls_data[i].reset_index(drop=True)
    if i == 0:
        # 特立尼达和多巴哥6月8日唯一的一名确诊患者治愈，R0为0。为消除后面对计算比值得影响，暂且将其取为1
        ls_data[i]['R0'][ls_data[i]['R0']=='0'] = 1
        # 对第0天的处理：由于敏感性分析的要求，要对阶段第1天前7天的R0取均值作为计算R0_Ratio的分母
        R0_AVG = pd.Series(np.zeros(Length))
        for pre in range(7):
            preindex = index - pre
            preindex = preindex.to_list()
            preR0 = data_prepare.loc[preindex,:]['R0']
            preR0[preR0=='#REF!'] = 1
            preR0 = preR0.reset_index(drop=True)
            preR0 = preR0.astype('float')
            R0_AVG += preR0
        R0_AVG = R0_AVG/7
        ls_data[i]['R0'] = R0_AVG
    R0_day0 = ls_data[0]['R0'].astype('float').reset_index(drop=True)
    R0_dayi = ls_data[i]['R0'].astype('float').reset_index(drop=True)
    ls_data[i]['R0_Ratio'] = R0_dayi/R0_day0
    # print(R0_dayi/R0_day0)
    ls_data[i].to_csv('data_day{}.csv'.format(i),index=False,sep=',',encoding="utf_8_sig")

# 此cell用于测试回归，用data_day1测试
i = 1
y_data = np.array(ls_data[i]['R0_Ratio'])
y_data = np.log(y_data)
y_data[y_data == -np.inf] = -5
print('leny=',y_data.shape)

ls_array = []
for j in range(1,9):
    ls_array.append(np.array(ls_data[i]['C{}_Introduce'.format(j)])[:,np.newaxis])
    ls_array.append(np.array(ls_data[i]['C{}_Lift'.format(j)])[:,np.newaxis])
ls_array.append(np.array(ls_data[i]['Multiple_Introduce'])[:,np.newaxis])
ls_array.append(np.array(ls_data[i]['Multiple_Lift'])[:,np.newaxis])
x_data = np.hstack(ls_array)
print(x_data.shape)

model = linear_model.LinearRegression()
model.fit(x_data, y_data)

for c in model.coef_:
    print(c)
print(type(model.coef_))
print(type(model.intercept_))
print("coefficients:",model.coef_)
print("intercept:",model.intercept_)

# 对每一天的数据对数回归
Introduce_coef = [[],[],[],[],[],[],[],[]]
Lift_coef = [[],[],[],[],[],[],[],[]]
Multiple_Introduce_coef = []
Multiple_Lift_coef = []
Intercepts = []
for i in range(1,29):
    # print(i)
    y_data = np.array(ls_data[i]['R0_Ratio'])
    # print(y_data[y_data<0])
    y_data = np.log(y_data)
    y_data[y_data == np.inf] = 5
    y_data[y_data == -np.inf] = -5
    # print(y_data.min())
    # print(y_data.max())
    # print(np.nan in y_data)

    ls_array = []
    for j in range(1,9):
        ls_array.append(np.array(ls_data[i]['C{}_Introduce'.format(j)])[:,np.newaxis])
        ls_array.append(np.array(ls_data[i]['C{}_Lift'.format(j)])[:,np.newaxis])
    ls_array.append(np.array(ls_data[i]['Multiple_Introduce'])[:,np.newaxis])
    ls_array.append(np.array(ls_data[i]['Multiple_Lift'])[:,np.newaxis])
    x_data = np.hstack(ls_array)

    model = linear_model.LinearRegression()
    model.fit(x_data, y_data)
    coef = list(model.coef_)
    intercept = float(model.intercept_)
    pd.DataFrame(coef).to_csv('coef_day{}.csv'.format(i),index=False,sep=',',encoding="utf_8_sig")
    
    for k in range(8):
        Introduce_coef[k].append(coef[2*k])
        Lift_coef[k].append(coef[2*k+1])
    Multiple_Introduce_coef.append(coef[-2])
    Multiple_Lift_coef.append(coef[-1])
    Intercepts.append(intercept)
    # print()
# print(Introduce_coef)
# print(Lift_coef)
# print(Multiple_Introduce_coef)
# print(Multiple_Lift_coef)
# print(Intercepts)

# 对每一天的数据做回归
Introduce_coef = [[],[],[],[],[],[],[],[]]
Lift_coef = [[],[],[],[],[],[],[],[]]
Multiple_Introduce_coef = []
Multiple_Lift_coef = []
Intercepts = []

for i in range(1,29):
    # print(i)
    y_data = np.array(ls_data[i]['R0_Ratio'])
    # print(y_data[y_data<0])
    y_data = np.log(y_data)
    y_data[y_data == np.inf] = 3
    y_data[y_data == -np.inf] = -3
    # print(y_data.min())
    # print(y_data.max())
    # print(np.nan in y_data)

    ls_array = []
    for j in range(1,9):
        ls_array.append(np.array(ls_data[i]['C{}_Introduce'.format(j)])[:,np.newaxis])
        ls_array.append(np.array(ls_data[i]['C{}_Lift'.format(j)])[:,np.newaxis])
    ls_array.append(np.array(ls_data[i]['Multiple_Introduce'])[:,np.newaxis])
    ls_array.append(np.array(ls_data[i]['Multiple_Lift'])[:,np.newaxis])
    x_data = np.hstack(ls_array)

    # pd.DataFrame(x_data).to_csv('x_data{}.csv'.format(i),index=False,sep=',',encoding="utf_8_sig")

    # 做20次回归，每次随机排除10个样本
    N_Sample = y_data.shape[0]
    coef = np.array([[]])
    intercept = np.array([])
    for turn in range(20):
        exclude_indexes = np.random.randint(0,N_Sample,10)
        exclude_indexes = np.unique(exclude_indexes)
        select_indexes = np.setdiff1d(np.arange(0,N_Sample,1),exclude_indexes)
        x_data_sample = x_data[select_indexes,:]
        y_data_sample = y_data[select_indexes]

        model = linear_model.LinearRegression()
        model.fit(x_data_sample, y_data_sample)
        coef_sample = model.coef_
        intercept_sample = float(model.intercept_)
        # print(i,turn,intercept_sample)
        if turn == 0:
            coef = coef_sample
        else:
            coef = np.vstack([coef,coef_sample])
        intercept = np.append(intercept,intercept_sample)
        # print(i,turn,intercept)
        # pd.DataFrame(x_data_sample).to_csv('x_sample{}.csv'.format(turn),index=False,sep=',',encoding="utf_8_sig")
    coef = coef.mean(axis=0)
    intercept = intercept.mean()
    #qpd.DataFrame(coef).to_csv('coef{}.csv'.format(i),index=False,sep=',',encoding="utf_8_sig")
    
    for k in range(8):
        Introduce_coef[k].append(coef[2*k])
        Lift_coef[k].append(coef[2*k+1])
    Multiple_Introduce_coef.append(coef[-2])
    Multiple_Lift_coef.append(coef[-1])
    Intercepts.append(intercept)
#print(Introduce_coef[0])
#print(Lift_coef[0])
#print(Intercepts)
print(N_Sample)

exclude_indexes = np.random.randint(0,10,5)
exclude_indexes = np.unique(exclude_indexes)
y_data = np.arange(0,100,1)
print(np.setdiff1d(y_data,exclude_indexes))
x_data = y_data.reshape(10,10)
y_data = y_data[exclude_indexes]
print(exclude_indexes)
print(y_data)
print(x_data)
x_data = x_data[exclude_indexes,:]
print(x_data)

#pd.DataFrame(Introduce_coef).to_csv('Introduce_coef.csv'.format(turn),index=False,sep=',',encoding="utf_8_sig")
#pd.DataFrame(Lift_coef).to_csv('Lift_coef.csv'.format(turn),index=False,sep=',',encoding="utf_8_sig")
#pd.DataFrame(Intercepts).to_csv('Intercepts.csv'.format(turn),index=False,sep=',',encoding="utf_8_sig")
pd.DataFrame(coef).to_csv('coef.csv'.format(turn),index=False,sep=',',encoding="utf_8_sig")

# R0和政策引入/取消关系
Interceptarray = np.array(Intercepts)
x = np.linspace(0,29,29)
for i in range(8):
    subfigure = plt.figure(figsize=(15,5))
    Introduce = np.array(Introduce_coef[i])
    Lift = np.array(Lift_coef[i])

    c1 = Introduce + Interceptarray
    c1 = np.hstack([np.array([0]),c1])
    lineIntroduce, = plt.plot(x,c1)
    plt.plot(x,c1,'bo')

    c2 = Lift + Interceptarray
    c2 = np.hstack([np.array([0]),c2])
    lineLift, = plt.plot(x,c2,'r')
    plt.plot(x,c2,'ro')

    c3 = Interceptarray
    c3 = np.hstack([np.array([0]),c3])
    lineIntercept, = plt.plot(x,c3,'y')
    plt.plot(x,c3,'yo')

    plt.title(ls_policy[i])
    plt.plot(x,np.zeros(29))
    plt.ylim(-0.2,0.2)
    plt.xlabel('时间')
    plt.ylabel('ln(R0)')
    plt.legend(handles=[lineIntroduce, lineLift, lineIntercept], labels=['引入此政策','放宽此政策','无政策（回归的截距）'],loc=2)
    plt.show()
